### Install necessary packages

In [1]:
# Install simpletransformers
# May require a runtime restart

!pip install simpletransformers

     |████████████████████████████████| 215kB 8.3MB/s 
     |████████████████████████████████| 2.1MB 14.1MB/s 
     |████████████████████████████████| 1.2MB 37.6MB/s 
     |████████████████████████████████| 204kB 44.4MB/s 
     |████████████████████████████████| 122kB 43.2MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 3.3MB 39.9MB/s 
     |████████████████████████████████| 8.2MB 19.0MB/s 
     |████████████████████████████████| 2.1MB 44.0MB/s 
     |████████████████████████████████| 81kB 7.8MB/s 
     |████████████████████████████████| 102kB 9.9MB/s 
     |████████████████████████████████| 133kB 47.7MB/s 
     |████████████████████████████████| 163kB 49.3MB/s 
     |████████████████████████████████| 245kB 50.7MB/s 
     |████████████████████████████████| 112kB 48.8MB/s 
     |████████████████████████████████| 112kB 49.7MB/s 
     |████████████████████████████████| 4.2MB 43.4MB/s 
     |████████████████████████████████| 81kB 9.1MB/s 


In [1]:
!pip install Unidecode

     |████████████████████████████████| 245kB 8.2MB/s 


In [2]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Load dataset

In [3]:
df = pd.read_csv("https://drive.google.com/uc?export=download&id=1b8MAiN-xBdk6scM-DnufkuijDZivZJqM")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  2400 non-null   object
 1   Polarity  2400 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 37.6+ KB


In [4]:
df.head()

,Sentence,Polarity
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [5]:
df['Polarity'].value_counts()

0    1213
1    1187
Name: Polarity, dtype: int64

In [6]:
df.shape

(2400, 2)

### Preprocessing


In [7]:
import unidecode

def preprocess(x):
  x = unidecode.unidecode(x)

  return x

df['Sentence'] = df['Sentence'].apply(preprocess) # Apply preprocessing
df.drop_duplicates('Sentence', inplace=True)  # Drop duplicates

In [8]:
df.shape

(2382, 2)

### Train-test split

In [9]:
from sklearn.model_selection import train_test_split

X = df[['Sentence', 'Polarity']]

X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_train['Polarity'].value_counts())

print(X_test.shape)
print(X_test['Polarity'].value_counts())

(1905, 2)
0    968
1    937
Name: Polarity, dtype: int64
(477, 2)
1    239
0    238
Name: Polarity, dtype: int64


### Modelling

In [10]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging
import sklearn

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [11]:
# Model configuration
model_args = ClassificationArgs(num_train_epochs=30, sliding_window=True, overwrite_output_dir=True, save_model_every_epoch=False, max_seq_length=400)

# Create a ClassificationModel
model = ClassificationModel("roberta", "roberta-base", args=model_args, use_cuda=True)


INFO:filelock:Lock 140625283407952 acquired on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

INFO:filelock:Lock 140625283407952 released on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock
INFO:filelock:Lock 140625359177296 acquired on /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7.lock


Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

INFO:filelock:Lock 140625359177296 released on /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7.lock
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of R

...

INFO:filelock:Lock 140625157811216 released on /root/.cache/huggingface/transformers/d3ccdbfeb9aaa747ef20432d4976c32ee3fa69663b379deb253ccfce2bb1fdc5.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab.lock
INFO:filelock:Lock 140625283569680 acquired on /root/.cache/huggingface/transformers/cafdecc90fcab17011e12ac813dd574b4b3fea39da6dd817813efa010262ff3f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock


Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

INFO:filelock:Lock 140625283569680 released on /root/.cache/huggingface/transformers/cafdecc90fcab17011e12ac813dd574b4b3fea39da6dd817813efa010262ff3f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock
INFO:filelock:Lock 140625157779792 acquired on /root/.cache/huggingface/transformers/d53fc0fa09b8342651efd4073d75e19617b3e51287c2a535becda5808a8db287.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730.lock


Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

INFO:filelock:Lock 140625157779792 released on /root/.cache/huggingface/transformers/d53fc0fa09b8342651efd4073d75e19617b3e51287c2a535becda5808a8db287.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730.lock


In [12]:
# Train the model
# Note: This is being trained on entire dataset

model.train_model(X, eval_df=X_test, f1 = sklearn.metrics.f1_score)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:449: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/2382 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 2382 features created from 2382 samples.


Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Running Epoch 0 of 30:   0%|          | 0/298 [00:00<?, ?it/s]

...

Running Epoch 27 of 30:   0%|          | 0/298 [00:00<?, ?it/s]

Running Epoch 28 of 30:   0%|          | 0/298 [00:00<?, ?it/s]

Running Epoch 29 of 30:   0%|          | 0/298 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.


(8940, 0.050735025861712135)

In [13]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(X_test, f1 = sklearn.metrics.f1_score)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1029: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/477 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 477 features created from 477 samples.


Running Evaluation:   0%|          | 0/60 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 1.0, 'tp': 239, 'tn': 238, 'fp': 0, 'fn': 0, 'f1': 1.0, 'eval_loss': 3.7702344419206687e-06}


### Test the model

In [14]:
# Load test data
df_test = pd.read_csv("https://drive.google.com/uc?export=download&id=1taoTluPBUMt9JkKAnlqDTrU49DJFpJGW")

In [15]:
df_test.head()

,Sentence,Polarity
0,A good commentary of today's love and undoubte...,1
1,For people who are first timers in film making...,1
2,"It was very popular when I was in the cinema, ...",1
3,It's a feel-good film and that's how I felt wh...,1
4,It has northern humour and positive about the ...,1


In [16]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  600 non-null    object
 1   Polarity  600 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 9.5+ KB


In [17]:
pred_test, res = model.predict(df_test['Sentence'].tolist(),multi_label=False)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/600 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 600 features created from 600 samples.


  0%|          | 0/75 [00:00<?, ?it/s]

In [18]:
my_submission = pd.DataFrame({'predicted': pred_test})
my_submission.head()

# This command will save the file to the local cloud instance; it will be deleted
# as soon as this Notebooks session ends.
#my_submission.to_csv('my_submission.csv', index=False)

,predicted
0,1
1,1
2,1
3,1
4,1


In [23]:
my_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   predicted  600 non-null    int64
dtypes: int64(1)
memory usage: 4.8 KB


In [19]:
# Download submission file
from google.colab import files
#files.download('my_submission.csv')

### Inspect wrong predictions
* Will need to use train-test split, not entire dataset, when training to get proper results

In [20]:
wrong_predictions

[]

In [21]:
predictions, raw_outputs = model.predict(['I really hated this movie. NOT!.'])
print(predictions)
print(raw_outputs)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/1 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 1 features created from 1 samples.


  0%|          | 0/1 [00:00<?, ?it/s]

[0]
[array([[ 6.18359375, -6.140625  ]])]


In [24]:
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


print("F1 Score   = {:.3f}".format(f1_score(df_test['Polarity'], pred_test, average="macro")))
print("Accuracy   = {:.3f}".format(accuracy_score(df_test['Polarity'], pred_test)))
print("AUC      = {:.3f}".format(roc_auc_score(df_test['Polarity'], pred_test)))
print("\nConfusion Matrix:")

print(confusion_matrix(y_true = df_test['Polarity'], y_pred = pred_test))

print("\n")

print("\nClassification Report:")

#class_names = [str(x) for x in search.best_estimator_.classes_]
print(classification_report(y_true = df_test['Polarity'], y_pred = pred_test))

F1 Score   = 0.930
Accuracy   = 0.930
AUC      = 0.931

Confusion Matrix:
[[271  16]
 [ 26 287]]



Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.94      0.93       287
           1       0.95      0.92      0.93       313

    accuracy                           0.93       600
   macro avg       0.93      0.93      0.93       600
weighted avg       0.93      0.93      0.93       600

